In [1]:
from graphdatascience import GraphDataScience
import pandas as pd

host = "bolt://localhost:7687"
user = "neo4j"
password = "sojongabcd"

gds = GraphDataScience(host, auth=(user, password))

/Users/kimnamju/Desktop/학교/4-1/소프트웨어종합설계(1)/NeoGPT-Explorer/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Joseon Relation 

In [5]:
import json

INPUT_FILE = '../data/sejong.json'

with open(INPUT_FILE) as file:
    nlp_output = json.load(file)

In [6]:
# Entity의 id(name) 중복되어서 안됨
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE e.id IS UNIQUE;
"""
)

""


In [4]:
nlp_import_query = """
UNWIND $data AS row
FOREACH (entity in row.entity | 
  MERGE (e:Entity {id: entity.name})
  ON CREATE SET e.type = entity.type,
                e.uri = entity.uri
)
WITH row
UNWIND row.fact AS fact
  WITH fact
  MERGE (source:Entity {id: fact.source.name})
  ON CREATE SET source.type = fact.source.type
  MERGE (target:Entity {id: fact.target.name})
  ON CREATE SET target.type = fact.target.type
  MERGE (source)-[:RELATIONSHIP {type: toUpper(replace(fact.relationship,' ','_'))}]-(target)
"""

# MERGE (source)-[:RELATIONSHIP]-(r:Relationship {type: toUpper(replace(fact.relationship,' ','_'))})-[:RELATIONSHIP]-(target)

batch_step = 30

for offset in range(0, len(nlp_output), batch_step):
    batch = nlp_output[offset : offset + batch_step]
    gds.run_cypher(nlp_import_query, {"data": batch})